In [1]:
import ultralytics
from ultralytics import YOLO
import os
import cv2
import time
import torch
import random
import shutil
import tqdm


In [2]:
print("CUDA Available: " + str(torch.cuda.is_available()))
print("Torch CUDA Version: " + str(torch.version.cuda))
# Check to make sure CUDA is available and does not say "None"
ultralytics.utils.checks.collect_system_info()

Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3080, 12045MiB)
Setup complete ✅ (24 CPUs, 62.7 GB RAM, 1616.8/1832.2 GB disk)



OS                  Linux-6.8.0-47-generic-x86_64-with-glibc2.35
Environment         Jupyter
Python              3.11.8
Install             pip
RAM                 62.66 GB
CPU                 12th Gen Intel Core(TM) i9-12900KF
CUDA                12.1

matplotlib          ✅ 3.8.3>=3.3.0
opencv-python       ✅ 4.9.0.80>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.12.0>=1.4.1
torch               ✅ 2.2.1>=1.8.0
torchvision         ✅ 0.17.1>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.8
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 2.2.1>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0


In [72]:
CURR_DIR = os.getcwd()
WORKSPACE_DIR = os.path.dirname(CURR_DIR)
DATASETS_DIR = WORKSPACE_DIR + '/datasets/'
DATA_YAML = WORKSPACE_DIR + '/data.yaml'

TEST_PATH = DATASETS_DIR + '/ims_2024_day6_run1/images/'

In [73]:
def test_model(model : YOLO, test_results_path: os.PathLike) -> None:
    """
    Test the fine-tuned model on test images and save the results.

    Parameters:
        model (YOLO): The fine-tuned YOLO model.
        test_results_path (os.PathLike): The path to save the test results.

    Returns:
        None

    """
    # Make sure the test save path exists
    if not os.path.exists(test_results_path):
        os.makedirs(test_results_path)

    false_pos = 0
    true_neg = 0

    # Inference fine-tuned model on test images and save results
    for file in os.listdir(TEST_PATH):
        file_path = os.path.join(TEST_PATH, file)
        output = model.predict(file_path)
        save_path = os.path.join(test_results_path, file)
        
        if len(output[0].boxes) == 0:
            true_neg += 1
        if len(output[0].boxes):
            false_pos += 1

    total_preds = true_neg + false_pos
    if total_preds > 0:
        precision = true_neg / (total_preds)
    print(total_preds)
    print(true_neg)
    print(false_pos)

    return true_neg, false_pos, precision

In [74]:
#initialize model
CURR_MODEL_PATH = WORKSPACE_DIR + '/models/yolo_model_files/epoch95.pt'
model = YOLO(CURR_MODEL_PATH)
data_dir = CURR_DIR + '/../data/'

# initialize test results
test_results_path = data_dir + '/test/annotation_results'
true_neg, false_pos, precision = test_model(model, test_results_path)



image 1/1 /home/annabelng/Desktop/YOLOv8-Fine-Tune/datasets/ims_2024_day6_run1/images/Image_0000004221_sec1725457888_nsec950296160.jpg: 800x1056 (no detections), 3.6ms
Speed: 2.1ms preprocess, 3.6ms inference, 0.2ms postprocess per image at shape (1, 3, 800, 1056)

image 1/1 /home/annabelng/Desktop/YOLOv8-Fine-Tune/datasets/ims_2024_day6_run1/images/Image_0000000091_sec1725457681_nsec146998183.jpg: 800x1056 (no detections), 3.4ms
Speed: 2.0ms preprocess, 3.4ms inference, 0.2ms postprocess per image at shape (1, 3, 800, 1056)

image 1/1 /home/annabelng/Desktop/YOLOv8-Fine-Tune/datasets/ims_2024_day6_run1/images/Image_0000001021_sec1725457727_nsec996251830.jpg: 800x1056 (no detections), 3.4ms
Speed: 2.0ms preprocess, 3.4ms inference, 0.2ms postprocess per image at shape (1, 3, 800, 1056)

image 1/1 /home/annabelng/Desktop/YOLOv8-Fine-Tune/datasets/ims_2024_day6_run1/images/Image_0000004016_sec1725457878_nsec700051258.jpg: 800x1056 (no detections), 3.4ms
Speed: 2.0ms preprocess, 3.4ms in

In [68]:
# ims day 5
print(f'True neg: {true_neg}, False_Pos: {false_pos}, Precision: {precision}')

True neg: 7186, False_Pos: 134, Precision: 0.9816939890710382


In [75]:
# ims day 6 run 1
print(f'True neg: {true_neg}, False_Pos: {false_pos}, Precision: {precision}')

True neg: 808, False_Pos: 1, Precision: 0.9987639060568603


In [71]:
# ims day 6 run 2
print(f'True neg: {true_neg}, False_Pos: {false_pos}, Precision: {precision}')

True neg: 2975, False_Pos: 93, Precision: 0.9696870925684485


In [ ]:
# ims day 6 run 1: 808 no detections, 1 car detection, 99.87% precise
# ims day 6 run 2: 2975 no detections, 98 car detections, 95.97% precise 
# ims day 5: 7186 no detections, 134 car detections, 98.17% precise